<a href="https://colab.research.google.com/github/AnabelBerumen/100DaysOfCode/blob/main/Data%20Cleaning/Pandas%20Cookbook/4_The_pandas_I_O_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [8]:
!python -m pip install adbc-driver-sqlite

In [9]:
import io
import pandas as pd
import sqlalchemy as sa
from adbc_driver_sqlite import dbapi
import timeit
import sqlalchemy as sa
import numpy as np

## 4 The pandas I/O System

## CSV – basic reading/writing

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame([
 ["Paul", "McCartney", 1942],
 ["John", "Lennon", 1940],
 ["Richard", "Starkey", 1940],
 ["George", "Harrison", 1943],
], columns=["first", "last", "birth"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [12]:
import io
buf = io.StringIO()

df.to_csv(buf)
print(buf.getvalue())

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943



In [13]:
buf.seek(0)
pd.read_csv(buf, dtype_backend='numpy_nullable')

,Unnamed: 0,first,last,birth
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940
2,2,Richard,Starkey,1940
3,3,George,Harrison,1943


In [14]:
buf.seek(0)
pd.read_csv(buf, dtype_backend='numpy_nullable', index_col=0)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [15]:
buf = io.StringIO()
df.to_csv(buf, index=False)
print(buf.getvalue())

first,last,birth
Paul,McCartney,1942
John,Lennon,1940
Richard,Starkey,1940
George,Harrison,1943



In [16]:
df = pd.DataFrame([
 ["McCartney, Paul", 1942],
 ["Lennon, John", 1940],
 ["Starkey, Richard", 1940],
 ["Harrison, George", 1943],
], columns=["name", "birth"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df

,name,birth
0,"McCartney, Paul",1942
1,"Lennon, John",1940
2,"Starkey, Richard",1940
3,"Harrison, George",1943


In [17]:
buf = io.StringIO()
df.to_csv(buf, index=False)
print(buf.getvalue())

name,birth
"McCartney, Paul",1942
"Lennon, John",1940
"Starkey, Richard",1940
"Harrison, George",1943



In [18]:
buf = io.StringIO()
df.to_csv(buf, index=False, sep='|')
print(buf.getvalue())

name|birth
McCartney, Paul|1942
Lennon, John|1940
Starkey, Richard|1940
Harrison, George|1943



In [19]:
df = pd.DataFrame({
 "col1": ["a"] * 1_000,
 "col2": ["b"] * 1_000,
 "col3": ["c"] * 1_000,
})
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df.head()

,col1,col2,col3
0,a,b,c
1,a,b,c
2,a,b,c
3,a,b,c
4,a,b,c


In [20]:
buf = io.BytesIO()
df.to_csv(buf, index=False, compression='gzip')
len(buf.getvalue())

69

In [21]:
df = pd.read_csv('/content/diamonds.csv', dtype_backend='numpy_nullable', nrows=1_000)
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/diamonds.csv'

In [ ]:
df.info()

In [ ]:
df['price'].describe()

In [ ]:
df['carat'].describe()

In [ ]:
df2 = pd.read_csv(
    '/content/diamonds.csv',
    nrows=1_000,
    dtype={
        'carat': pd.Float32Dtype(),
        'cut': pd.StringDtype(),
        'color': pd.StringDtype(),
        'clarity': pd.StringDtype(),
        'depth': pd.Float32Dtype(),
        'table': pd.Float32Dtype(),
        'price': pd.Int16Dtype(),
        'x': pd.Float32Dtype(),
        'y': pd.Float32Dtype(),
        'z': pd.Float32Dtype(),
    }
)
df2.info()

In [ ]:
df.describe()

In [ ]:
df2.describe()

In [ ]:
df2['cut'].unique()

In [ ]:
df2['color'].unique()

In [ ]:
df2['clarity'].unique()

In [ ]:
df3 = pd.read_csv(
    '/content/diamonds.csv',
    nrows=1_000,
    dtype={
        'carat': pd.Float32Dtype(),
        'cut': pd.StringDtype(),
        'color': pd.StringDtype(),
        'clarity': pd.StringDtype(),
        'depth': pd.Float32Dtype(),
        'table': pd.Float32Dtype(),
        'price': pd.Int16Dtype(),
        'x': pd.Float32Dtype(),
        'y': pd.Float32Dtype(),
        'z': pd.Float32Dtype(),
    }
)
cat_cols = ['cut', 'color', 'clarity']
df3[cat_cols] = df[cat_cols].astype(pd.CategoricalDtype())
df3.info()

In [ ]:
dtypes={
        'carat': pd.Float32Dtype(),
        'cut': pd.StringDtype(),
        'color': pd.StringDtype(),
        'clarity': pd.StringDtype(),
        'depth': pd.Float32Dtype(),
        'table': pd.Float32Dtype(),
        'price': pd.Int16Dtype(),
}

df4 = pd.read_csv(
    "/content/diamonds.csv",
    nrows=1_000,
    dtype = dtypes,
    usecols = dtypes.keys(),
    )

cat_cols = ['cut', 'color', 'clarity']
df4[cat_cols] = df4[cat_cols].astype(pd.CategoricalDtype())
df4.info()

In [ ]:
dtypes = { # does not include x, y, or z
    "carat": pd.Float32Dtype(),
    "cut": pd.StringDtype(),
    "color": pd.StringDtype(),
    "clarity": pd.StringDtype(),
    "depth": pd.Float32Dtype(),
    "table": pd.Float32Dtype(),
    "price": pd.Int16Dtype(),
}
df_iter = pd.read_csv(
    "/content/diamonds.csv",
    nrows=1_000,
    dtype=dtypes,
    usecols=dtypes.keys(),
    chunksize=200
)

for df in df_iter:
    cat_cols = ['cut', 'color', 'clarity']
    df[cat_cols] = df[cat_cols].astype(pd.CategoricalDtype())
    print(f'processed chunk of shape {df.shape}')

In [ ]:
# if you want to read only the Categorical columns
def startwith_c(column_name: str) ->bool:
    return column_name.startswith('c')

pd.read_csv(
    '/content/diamonds.csv',
    dtype_backend='numpy_nullable',
    usecols=startwith_c,
)

## Microsoft Excel - Basic reading/writing

install python -m pip install openpyxl

In [ ]:
!python -m pip install openpyxl

In [ ]:
df = pd.DataFrame([
 ["Paul", "McCartney", 1942],
 ["John", "Lennon", 1940],
 ["Richard", "Starkey", 1940],
 ["George", "Harrison", 1943],
], columns=["first", "last", "birth"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df

In [ ]:
import io
buf = io.BytesIO()
df.to_excel(buf)

In [ ]:
buf.seek(0)
pd.read_excel(buf, dtype_backend='numpy_nullable')

In [ ]:
buf.seek(0)
pd.read_excel(buf, dtype_backend='numpy_nullable', index_col=0)

In [ ]:
buf = io.BytesIO()
df.to_excel(buf, index=False)
buf.seek(0)
pd.read_excel(buf, dtype_backend='numpy_nullable')

In [ ]:
buf.seek(0)
dtypes = {
    'first': pd.StringDtype(),
    'last': pd.StringDtype(),
    'birth': pd.Int16Dtype(),
}

df = pd.read_excel(buf, dtype=dtypes)
df.dtypes

Workbook where Sheet1 contains no useful data

Workbook where another sheet has relevant data

In [ ]:
pd.read_excel(
    '/content/beatles.xlsx',
    dtype_backend='numpy_nullable',
    sheet_name='the_data',
    skiprows=4,
    usecols='C:E',
)

In [ ]:
pd.read_excel(
    "/content/beatles.xlsx",
    dtype_backend="numpy_nullable",
    sheet_name="the_data",
    skiprows=4,
    usecols=["first", "last", "birth"],
)

In [ ]:
df = pd.read_excel(
    '/content/hierarchical.xlsx',
    dtype_backend='numpy_nullable',
    index_col=[0,1],
    header=[0, 1],
)

df

“What does the Q2 performance look like year over year for every East Sub-Region?”

In [ ]:
df.loc[(slice(None), 'East'), (slice(None), 'Q2')]

## SQL using SQLAlchemy

install SQLAlchemy

In [59]:
!python -m pip install sqlalchemy

In [60]:
import sqlalchemy as sa
engine = sa.create_engine('sqlite:///:memory:')

In [61]:
df = pd.DataFrame([
    ['dog', 4],
    ['cat', 4],
], columns=['animal', 'num_legs'])
df = df.convert_dtypes(dtype_backend='numpy_nullable')

df.to_sql('table_name', engine, index=False)

2

In [62]:
pd.read_sql('table_name', engine, dtype_backend='numpy_nullable')

,animal,num_legs
0,dog,4
1,cat,4


In [63]:
pd.read_sql(
    'SELECT SUM(num_legs) AS total_legs FROM table_name',
    engine,
    dtype_backend = 'numpy_nullable'
)

,total_legs
0,8


In [66]:
df = pd.DataFrame([
    ['dog', 4],
    ['cat', 4],
    ['human', 2],
], columns = ['animal', 'num_legs'])
df = df.convert_dtypes(dtype_backend='numpy_nullable')
df.to_sql('table_name', engine, index=False, if_exists='replace')

3

In [67]:
new_data = pd.DataFrame([['centipede', 100]], columns=['animal', 'num_legs'])
new_data.to_sql('table_name', engine, index=False, if_exists='append')
pd.read_sql('table_name', engine, dtype_backend='numpy_nullable')

,animal,num_legs
0,dog,4
1,cat,4
2,human,2
3,centipede,100


## SQL using ADBC

In [ ]:
!python -m pip install adbc-driver-sqlite

In [ ]:
from adbc_driver_sqlite import dbapi
df = pd.DataFrame([
    ['dog', 4],
    ['cat', 4],
    ['human', 2],
], columns=['animal', 'num_legs'])
df = df.convert_dtypes(dtype_backend='numpy_nullable')
df

In [ ]:
with dbapi.connect('file::memory:') as conn:
    df.to_sql('table_name', conn, index=False, if_exists='replace')
    df = pd.read_sql(
        "SELECT * FROM table_name",
        conn,
        dtype_backend='numpy_nullable',
    )
df

In [ ]:
import timeit
import sqlalchemy as sa
import numpy as np
from adbc_driver_sqlite import dbapi

In [ ]:
np.random.seed(42)
df = pd.DataFrame(
    np.random.randn(10_000, 10),
    columns=list('abcdefghij')
)

with sa.create_engine('sqlite:///:memory:').connect() as conn:
    func = lambda: df.to_sql('test_table', conn, if_exists='replace')
    print(timeit.timeit(func, number=100))

In [ ]:
with dbapi.connect('file::memory') as conn:
    func = lambda :  df.to_sql('test_table', conn, if_exists='replace')
    print(timeit.timeit(func, number=100))

## Apache Parquet
* Metadata Storage
* Partitioning
* Query Support
* Parallelization
* Compactness

In [22]:
import io
buf = io.BytesIO()
df = pd.DataFrame([
    ["Paul", "McCartney", 1942],
    ["John", "Lennon", 1940],
    ["Richard", "Starkey", 1940],
    ["George", "Harrison", 1943],
    ], columns=["first", "last", "birth"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [24]:
df.to_parquet(buf, index=False)
buf.seek(0)
pd.read_parquet(buf)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [25]:
buf = io.BytesIO()
df.to_parquet(buf, index=False)
buf.seek(0)
pd.read_parquet(buf).dtypes

,0
first,string[python]
last,string[python]
birth,Int64


In [39]:
suboptimal_df = pd.DataFrame([
    [0, 'foo'],
    [1, 'bar'],
    [2, 'baz']
], columns=['int_col', 'str_col'])
buf = io.BytesIO()
suboptimal_df.to_parquet(buf, index=False)
buf.seek(0)
pd.read_parquet(buf, dtype_backend='numpy_nullable').dtypes

,0
int_col,Int64
str_col,string[python]


In [40]:
path = 'C://Users//Equipo//Desktop//ciencia_de_datos//workout//Pandas_Cookbook//data//partitions'
filedir = "//2022//q1_sales.parquet"
fullpath = path + filedir
pd.read_parquet(
    fullpath,
    )

FileNotFoundError: /content/C://Users//Equipo//Desktop//ciencia_de_datos//workout//Pandas_Cookbook//data//partitions//2022//q1_sales.parquet

125 pag